##**Importação das bibliotecas**

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/trab_IA/dataset_espectro.zip

Archive:  /content/drive/MyDrive/trab_IA/dataset_espectro.zip
   creating: dataset_espectro/
   creating: dataset_espectro/test/
   creating: dataset_espectro/test/Blues/
  inflating: dataset_espectro/test/Blues/108.png  
   creating: dataset_espectro/test/Classical/
  inflating: dataset_espectro/test/Classical/59.png  
   creating: dataset_espectro/test/Country/
  inflating: dataset_espectro/test/Country/93.png  
   creating: dataset_espectro/test/Eletronical/
  inflating: dataset_espectro/test/Eletronical/47.png  
   creating: dataset_espectro/test/Folk/
  inflating: dataset_espectro/test/Folk/75.png  
   creating: dataset_espectro/test/Hip Hop/
  inflating: dataset_espectro/test/Hip Hop/62.png  
   creating: dataset_espectro/test/Instrumental/
  inflating: dataset_espectro/test/Instrumental/6.png  
   creating: dataset_espectro/test/Jazz/
  inflating: dataset_espectro/test/Jazz/87.png  
   creating: dataset_espectro/test/Pop/
  inflating: dataset_espectro/test/Pop/33.png  
   creati

##**Criação da Rede Neural Convolucional**

###**Inicializando a CNN**

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator

##Pré-processamento


In [ ]:
train_dir = "/content/dataset_espectro/train"
test_dir = "/content/dataset_espectro/test"
val_dir = "/content/dataset_espectro/val"

In [ ]:
train_datagen = ImageDataGenerator(rescale=1.0 / 255)
test_datagen = ImageDataGenerator(rescale=1.0 / 255)
val_datagen = ImageDataGenerator(rescale=1.0 / 255)

In [ ]:
train_gen = train_datagen.flow_from_directory(
    train_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 77 images belonging to 11 classes.


In [ ]:
val_gen = val_datagen.flow_from_directory(
    val_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 22 images belonging to 11 classes.


In [ ]:
test_gen = test_datagen.flow_from_directory(
    test_dir,
    target_size=(400,1000),
    batch_size=32,
    class_mode='categorical'
)

Found 11 images belonging to 11 classes.


In [ ]:
input_shape = [400,1000,3]
num_classes = len(train_gen.class_indices)

##O modelo em si


In [ ]:
cnn = tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))

In [ ]:
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

###**Treinando a CNN e estimando no conjunto de testes**

In [ ]:
history = cnn.fit(train_gen, epochs=30, validation_data=val_gen)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


3/3 ━━━━━━━━━━━━━━━━━━━━ 90s 36s/step - accuracy: 0.1189 - loss: 39.5148 - val_accuracy: 0.0909 - val_loss: 14.1094
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 83s 25s/step - accuracy: 0.0637 - loss: 16.3326 - val_accuracy: 0.0909 - val_loss: 2.4451
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 145s 35s/step - accuracy: 0.0491 - loss: 2.6196 - val_accuracy: 0.0909 - val_loss: 2.3553
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 35s/step - accuracy: 0.1622 - loss: 2.3842 - val_accuracy: 0.0909 - val_loss: 2.3484
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 86s 35s/step - accuracy: 0.1425 - loss: 2.3301 - val_accuracy: 0.0909 - val_loss: 2.2953
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 141s 26s/step - accuracy: 0.1685 - loss: 2.1613 - val_accuracy: 0.2727 - val_loss: 2.2871
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 85s 26s/step - accuracy: 0.2581 - loss: 2.0095 - val_accuracy: 0.2273 - val_loss: 2.2197
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 140s 25s/step - accuracy: 0.3524 - loss: 1.7990 - val_accuracy: 0.2727 - val_loss: 2.1360
Epoc

In [ ]:
test_loss, test_accuracy = cnn.evaluate(test_gen)
print(f"Perda nos dados de teste: {test_loss}")
print(f"Acurácia nos dados de teste: {test_accuracy}")

NameError: name 'cnn' is not defined

In [ ]:
# Processar a imagem
from tensorflow.keras.preprocessing import image
import numpy as np


test_image = image.load_img("/content/dataset_espectro/test/Eletronical/47.png", target_size=(400, 1000))  # Tamanho da imagem usado no treinamento
test_image = image.img_to_array(test_image)  # Converte a imagem para um array NumPy
test_image = np.expand_dims(test_image, axis=0)  # Adiciona uma dimensão para o batch
test_image = test_image

In [ ]:
result = cnn.predict(test_image)

print(f"Result: {result}")

predicted_class_index = np.argmax(result)  # Obtém o índice da classe com maior probabilidade
print(f"Predicted class index: {predicted_class_index}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step
Result: [[1.0000000e+00 0.0000000e+00 3.3637510e-26 2.7023976e-25 0.0000000e+00
  6.4744788e-27 4.5108618e-16 0.0000000e+00 4.1161588e-12 3.1795836e-29
  8.2470836e-11]]
Predicted class index: 0


In [ ]:
# Dicionário de classes (gerado no treinamento)
class_indices = {'rock': 0, 'jazz': 1, 'pop': 2, 'hip hop':3,'blues':4,'classical':5,'country':6,'eletronical':7,'folk':8,'instrumental':9,'soul':10}  # Substitua com as classes reais do treinamento
index_to_class = {v: k for k, v in class_indices.items()}  # Converte índice para nome da classe

# Obter o nome da classe prevista
prediction = index_to_class[predicted_class_index]

# Exibir o resultado
print(f"Predição: {prediction}")

Predição: rock


# Função para receber o gênero de uma música do dataset de acordo com seu ID

In [46]:
import pandas as pd

def check_gender_by_id(filename, id, genders) -> None | str:
    df = pd.read_excel(filename)

    genero = df.loc[df['id'] == id, 'genero']

    if not genero.empty:
        if genero.iloc[0] in genders:
            return genero.iloc[0]
        else:
            return None
    else:
        return "ID not found"

In [47]:
# Exemplo de uso
arquivo = '/content/generos.xlsx'
id_musica = 17  # Coloque o ID desejado
genders = ['Rock', 'Jazz', 'Pop', 'Classical', 'Country', 'Eletronical']
genero = check_gender_by_id(arquivo, id_musica, genders)
print(f'O gênero da música com ID {id_musica} é: {genero}')

O gênero da música com ID 17 é: Rock


In [48]:
def dividir_musicas_em_listas(arquivo_xlsx, generos_validos):
    # Carrega o arquivo Excel para um DataFrame
    df = pd.read_excel(arquivo_xlsx)

    # Dicionário para armazenar as músicas de cada gênero
    generos = {}

    # Itera por todas as músicas no arquivo
    for id_musica in df['id']:
        # Usa a função check_gender_by_id para obter o gênero da música
        genero = check_gender_by_id(arquivo_xlsx, id_musica, generos_validos)

        # Se o gênero for válido (não None e dentro da lista de gêneros válidos), adiciona o ID
        if genero and genero in generos_validos:
            if genero not in generos:
                generos[genero] = []
            generos[genero].append(id_musica)

    # Listas para armazenar as músicas de treino, validação e teste
    musicas_treino = []
    musicas_validacao = []
    musicas_teste = []

    # Agora, vamos dividir as músicas por gênero em treino, validação e teste de forma aleatória
    for genero, musicas in generos.items():
        # Embaralha as músicas para aleatoriedade
        random.shuffle(musicas)

        # Verifica se há pelo menos 30 músicas para dividir
        if len(musicas) >= 30:
            # Divide as músicas de forma aleatória
            treino = musicas[:21]
            validacao = musicas[21:27]
            teste = musicas[27:30]

            # Adiciona as listas para cada categoria
            musicas_treino.extend(treino)
            musicas_validacao.extend(validacao)
            musicas_teste.extend(teste)
        else:
            # Se houver menos de 30 músicas, podemos decidir o que fazer, mas aqui estamos ignorando as músicas
            pass

    # Retorna as 3 listas
    return musicas_treino, musicas_validacao, musicas_teste

In [49]:
import random
treino,validacao,teste = dividir_musicas_em_listas('/content/generos.xlsx',genders)

In [ ]:
for x in treino:
  print(check_gender_by_id('/content/generos.xlsx',x,genders))

Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Rock
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Pop
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Eletronical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Classical
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Jazz
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country
Country


In [50]:
!unzip "/content/espectrogramas.zip"

Archive:  /content/espectrogramas.zip
replace espectrogramas/238.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: espectrogramas/238.png  
  inflating: espectrogramas/182.png  
  inflating: espectrogramas/150.png  
  inflating: espectrogramas/15.png   
  inflating: espectrogramas/130.png  
  inflating: espectrogramas/138.png  
  inflating: espectrogramas/231.png  
  inflating: espectrogramas/208.png  
  inflating: espectrogramas/249.png  
  inflating: espectrogramas/88.png   
  inflating: espectrogramas/216.png  
  inflating: espectrogramas/180.png  
  inflating: espectrogramas/178.png  
  inflating: espectrogramas/190.png  
  inflating: espectrogramas/206.png  
  inflating: espectrogramas/93.png   
  inflating: espectrogramas/137.png  
  inflating: espectrogramas/115.png  
  inflating: espectrogramas/28.png   
  inflating: espectrogramas/23.png   
  inflating: espectrogramas/84.png   
  inflating: espectrogramas/118.png  
  inflating: espectrogramas/52.png   
  inflating: esp

In [51]:
# Exemplo: Gêneros associados aos IDs
id_to_genre = {
    1: "rock",
    2: "pop",
    3: "jazz",
    4: "eletronical",
    5: "classical",
    6: "country"
    # Adicione todos os IDs e seus gêneros
}

# Converta os gêneros para índices numéricos para o treinamento
genre_to_index = {genre: idx for idx, genre in enumerate(set(id_to_genre.values()))}

# Exemplo: rock -> 0, pop -> 1, jazz -> 2
id_to_label = {id: genre_to_index[genre] for id, genre in id_to_genre.items()}


In [52]:
import tensorflow as tf
import os


base_dir = "/content/espectrogramas"

def id_to_filepath(id):
  return os.path.join(base_dir,f"{id}.png")

In [56]:
df = pd.read_excel('/content/generos.xlsx')

# Cria o mapeamento de gêneros para índices
genre_to_index = {genre: idx for idx, genre in enumerate(set(df['genero']))}

# Mapeia IDs das músicas aos índices das classes
id_to_label = {row['id']: genre_to_index[row['genero']] for _, row in df.iterrows()}

print("id_to_label:", id_to_label)

id_to_label: {1: 1, 2: 1, 3: 1, 4: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1, 10: 1, 11: 6, 12: 6, 13: 6, 14: 6, 15: 6, 16: 6, 17: 6, 18: 6, 19: 6, 20: 6, 21: 7, 22: 7, 23: 7, 24: 7, 25: 7, 26: 7, 27: 7, 28: 7, 29: 7, 30: 7, 31: 4, 32: 4, 33: 4, 34: 4, 35: 4, 36: 4, 37: 4, 38: 4, 39: 4, 40: 4, 41: 2, 42: 2, 43: 2, 44: 2, 45: 2, 46: 2, 47: 2, 48: 2, 49: 2, 50: 2, 51: 5, 52: 5, 53: 5, 54: 5, 55: 5, 56: 5, 57: 5, 58: 5, 59: 5, 60: 5, 61: 3, 62: 3, 63: 3, 64: 3, 65: 3, 66: 3, 67: 3, 68: 3, 69: 3, 70: 3, 71: 10, 72: 10, 73: 10, 74: 10, 75: 10, 76: 10, 77: 10, 78: 10, 79: 10, 80: 10, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 8, 92: 8, 93: 8, 94: 8, 95: 8, 96: 8, 97: 8, 98: 8, 99: 8, 100: 8, 101: 9, 102: 9, 103: 9, 104: 9, 105: 9, 106: 9, 107: 9, 108: 9, 109: 9, 110: 9, 111: 1, 112: 1, 113: 1, 114: 1, 115: 1, 116: 1, 117: 1, 118: 1, 119: 1, 120: 1, 121: 1, 122: 1, 123: 1, 124: 1, 125: 1, 126: 1, 127: 1, 128: 1, 129: 1, 130: 1, 131: 6, 132: 6, 133: 6, 134: 6, 135: 6, 136:

In [57]:
def create_tf_dataset_with_labels(id_list, id_to_label):
    filepaths = [id_to_filepath(id) for id in id_list]
    labels = [id_to_label[id] for id in id_list]

    # Cria um dataset contendo pares (caminho, rótulo)
    dataset = tf.data.Dataset.from_tensor_slices((filepaths, labels))

    # Carrega a imagem e retorna (imagem, rótulo)
    def load_image_and_label(filepath, label):
        image = tf.io.read_file(filepath)
        image = tf.image.decode_jpeg(image, channels=3)
        image = tf.image.resize(image, [224, 224])  # Tamanho esperado pela CNN
        return image, label

    dataset = dataset.map(load_image_and_label, num_parallel_calls=tf.data.AUTOTUNE)
    return dataset


In [58]:
print("IDs no treino:", treino)
print("IDs no val:", validacao)
print("IDs no teste:", teste)

# IDs presentes no dicionário de mapeamento
print("IDs no id_to_label:", list(id_to_label.keys()))

IDs no treino: [147, 137, 16, 13, 17, 146, 143, 15, 142, 145, 133, 14, 132, 131, 141, 139, 19, 135, 144, 18, 12, 168, 39, 151, 35, 34, 161, 155, 167, 152, 160, 163, 166, 165, 153, 170, 162, 169, 156, 157, 158, 33, 176, 171, 183, 180, 49, 186, 185, 190, 187, 42, 45, 177, 46, 189, 173, 182, 50, 41, 184, 44, 175, 197, 198, 57, 52, 60, 196, 59, 202, 204, 53, 201, 209, 192, 194, 200, 51, 210, 55, 191, 54, 199, 218, 89, 215, 213, 220, 81, 83, 217, 87, 90, 211, 224, 228, 230, 88, 86, 227, 212, 82, 225, 223, 250, 234, 238, 241, 232, 248, 92, 100, 246, 239, 99, 235, 236, 96, 245, 249, 91, 97, 247, 237, 242]
IDs no val: [140, 20, 136, 148, 138, 149, 37, 164, 32, 38, 36, 154, 181, 174, 172, 188, 179, 47, 58, 203, 207, 193, 206, 56, 226, 214, 216, 221, 84, 219, 240, 94, 244, 231, 95, 98]
IDs no teste: [150, 134, 11, 40, 31, 159, 178, 43, 48, 205, 195, 208, 85, 229, 222, 243, 233, 93]
IDs no id_to_label: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26,

In [59]:
train_dataset = create_tf_dataset_with_labels(treino, id_to_label)
val_dataset = create_tf_dataset_with_labels(validacao, id_to_label)
test_dataset = create_tf_dataset_with_labels(teste, id_to_label)

# Agrupar em lotes e otimizar
batch_size = 32
train_dataset = train_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)


In [66]:
import cv2 as cv

img = cv.imread("/content/espectrogramas/125.png")


input_shape = img.shape

print(input_shape)

(924, 2325, 3)


In [67]:
cnn = tf.keras.models.Sequential()
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3, activation='relu', input_shape=input_shape))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Conv2D(filters=128,kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size = 2))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))
cnn.add(tf.keras.layers.Dropout(0.5))
cnn.add(tf.keras.layers.Dense(units=num_classes, activation='softmax'))
cnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


NameError: name 'num_classes' is not defined